In [ ]:
!unzip test_cartoon.zip # cartoonized

In [ ]:
!unzip test.zip    # original

In [4]:
from keras.layers import Conv2D
from keras.layers import UpSampling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import RepeatVector
from keras.layers import Reshape
from keras.layers.merge import concatenate
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from skimage import io, color
from os import listdir
import numpy as np

In [ ]:
def define_model_baseline(img_h, img_w):

    # encoder
    inputs = Input(shape=(img_h, img_w, 3,))
    encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(inputs)
    encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
    encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
    encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
    
    # decoder
    
    decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)
    decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)
    decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
    decoder_output = Conv2D(3, (3, 3), activation='tanh', padding='same')(decoder_output)
    decoder_output = UpSampling2D((2, 2))(decoder_output)


    # tie it together 
    model = Model(inputs=inputs, outputs=decoder_output)
    model.compile(loss='mse', optimizer='adam', metrics=['acc'])
    return model

In [ ]:
model=define_model_baseline(256, 256)

In [ ]:
print(model.summary())

In [ ]:
training_dir = '../content/cartoon/'

Orig = []
BW = []
for filename in listdir(training_dir):
  data_orig = img_to_array(load_img(training_dir+filename, target_size=(256,256)))
  data_orig = 1.0/255*data_orig
  data_BW = img_to_array(load_img(training_dir+filename, target_size=(256,256), color_mode='grayscale'))
  data_BW = np.reshape(data_BW, newshape=(256,256))
  data_BW_3 = np.zeros((256, 256, 3))
  data_BW_3[:,:,0] = data_BW 
  data_BW_3[:,:,1] = data_BW
  data_BW_3[:,:,2] = data_BW
  data_BW_3 = 1.0/255*data_BW_3
  Orig.append(data_orig)
  BW.append(data_BW_3)
  
Orig = np.array(Orig)
BW = np.array(BW)

In [ ]:
train_orig, _, train_BW, _ = train_test_split(Orig, BW, test_size=0.2, random_state=42)

In [ ]:
fit_history = model.fit(train_BW, train_orig, epochs=15, verbose=1)

In [ ]:
model.save('model.h5')

In [ ]:
A = []
B = []
for img in BW:
  cartoon_rgb = model.predict(img)
  #
  cartoon_rgb = 1.0/255*cartoon_rgb
  #
  cartoon_lab = color.rgb2lab(cartoon_rgb)
  a = cartoon_lab[:, :, 1]
  b = cartoon_lab[:, :, 2]
  A.append(a)
  B.append(b)


original_dir = '../content/test/'

L = []
for filename in listdir(training_dir):
  data_rgb = img_to_array(load_img(original_dir+filename, target_size=(256,256)))
  data_rgb = 1.0/255*data_rgb
  data_lab = color.rgb2lab(data_rgb)
  l = data_lab[:, :, 0]
  L.append(l)

In [ ]:
results_dir = '../content/results/'

results = np.empty((len(L), 256, 256, 3))
img = 0
for filename in listdir(training_dir):
  results[img,:,:,0] = L[img]
  results[img,:,:,1] = A[img]
  results[img,:,:,2] = B[img]
  results[img,:,:,:] = color.lab2rgb(results[img,:,:,:])
  io.imsave(results_dir+filename, results[img,:,:,:])
  img += 1

In [ ]:
!zip -r results.zip results

In [ ]:
from google.colab import files
files.download("results.zip")

1. loading immagini originali e cartoon 
  - resized 256x256

2. fittare su: 
  - input -> train cartoon BW
  - target -> train cartoon orig

3. testare su tutto dataset cartoon

4. salva immagini cartoon ricolorate

5. estrarre 'l' da immagini originali + 'ab' da immagini cartoon ricolorate

6. salva immagini ottenute

7. calcola metriche